In [1]:
import torch
import torch.nn as nn

import sys
sys.path.insert(0,'..')
from utils.checkpoint import CheckPointer
from models.encoder import Encoder
from models.decoder import FrameLSTM, EventLSTM
from models.attention import Attention1,Attention2
from models.models import Model3

from collections import defaultdict, OrderedDict
import matplotlib.pyplot as plt
from matplotlib.image import imread
import numpy as np
import glob
import os

from PIL import Image
import torch
import torchvision.transforms as transforms
from tqdm import tqdm
import glob
import pickle
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'utils'

In [ ]:
joint_details = {1 : 'HEAD',
                 2 : 'NECK',
                 3 : 'RIGHT_SHOULDER',
                 4 : 'RIGHT_ELBOW',
                 5 : 'RIGHT_HAND',
                 6 : 'LEFT_SHOULDER',
                 7 : 'LEFT_ELBOW',
                 8 : 'LEFT_HAND',
                 9 : 'RIGHT_HIP',
                 10: 'RIGHT_KNEE',
                 11: 'RIGHT_FOOT'
                 12: 'LEFT_HIP',
                 13: 'LEFT_KNEE',
                 14: 'LEFT_FOOT',
                 15: 'HOCKEY GRIP'
                 16: 'HOCKEY Heel',
                 }

connect_map = [[1,2,2,2,2,3,4,6,7,9,10,12,13,15],[2,3,9,12,6,4,5,7,8,10,11,13,14,16]]


In [ ]:
def plot_frames(frames_code,reqd_pose_data, frames_pth, acts):
    
    frames_code = [int(x) for x in frames_code.strip().split('-')]
    
    for idx,row in enumerate(reqd_pose_data):
        posture = row

        posture_data = [x.strip() for x in posture.split(',')]

        joint_info = {}
        for i, n in enumerate(range(1, len(posture_data), 3)):

            x = 640 - float(posture_data[n])*640
            y = 480 - (float(posture_data[n+1])/1)*480
            z = posture_data[n+2]

            joint_info[i+1] = [x, y, z]

        person_1 = {k:joint_info[k] for k in range(1,16,1)}
        #print(person_1)
        person_2 = {k-15:joint_info[k] for k in range(16,31,1)}
        # print(person_1)

        count1 = 15
        for i, (x,y,z) in list(person_1.items()):
            if x<0 or x>640 or y<0 or y>480:
                    count1 -= 1

        count2 = 15
        for i, (x,y,z) in list(person_2.items()):
            if x<0 or x>640 or y<0 or y>480:
                    count2 -= 1
                    
        wt1 , wt2 = acts[idx][0], acts[idx][1]
                    
#         if frames_code[2]==2:
#             wt1, wt2 = wt2, wt1

        print(wt1, wt2)
        for key, value in person_1.items():
            plt.plot(value[0], value[1], 'bo', markersize=2)
        for m, n in zip(connect_map[0], connect_map[1]):
            plt.plot((person_1[m][0], person_1[n][0]), (person_1[m][1], person_1[n][1]), 'b--', linewidth=5*wt1)

        for key, value in person_2.items():
            plt.plot(value[0], value[1], 'go', markersize=2)
        for m, n in zip(connect_map[0], connect_map[1]):
            plt.plot((person_2[m][0], person_2[n][0]), (person_2[m][1], person_2[n][1]), 'g--', linewidth=5*wt2)

        plt.xlim(0, 640)
        plt.ylim(0, 480)


        img = np.array(plt.imread(frames_pth[idx]))
        img = img[::-1,:,:]
        plt.imshow(img)
        plt.pause(0.1)
        plt.clf()

In [2]:
#workdir/1/1
model_pth = '/home/fay/Desktop/Key-Actor-Detection/work_dir/model3axwd/1/0a38f1849b4543eab0388493140430ea/checkpoints/epoch=14-val_acc_epoch=0.6210.ckpt'

model = Model3(eventLSTM = {'hidden_size':256}, pose_dim=30, num_classes=8, attention_type=2)
ckpt = CheckPointer([model])
ckpt.load_checkpoint(model_pth)

model.cuda()

NameError: name 'Model3' is not defined